In [6]:
# Cell 0 — pull Phase-1 into this notebook
LIGHT_IMPORT = True

# If the file is in the current folder:
%run -i "Calc_Embeddings_Model Training.ipynb"

# quick sanity checks
print("df shape:", df.shape if 'df' in globals() else 'df missing')
print("have get_matches?", 'get_matches' in globals())

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

In [7]:
import numpy as np
import pandas as pd

In [8]:
def _ensure_text_sim(cands: pd.DataFrame) -> pd.DataFrame:
    c = cands.copy()
    if "text_sim" not in c.columns and "score" in c.columns:
        c["text_sim"] = c["score"]
    c["text_sim"] = c["text_sim"].fillna(0).clip(lower=0)
    return c

In [9]:
needed = ["top_k_semantic_neighbors_by_text","filter_by_orientation","filter_by_location",
          "filter_by_age","filter_by_diet","filter_by_drinks","filter_by_smokes","filter_by_drugs",
          "compute_component_scores","apply_weights","get_matches"]
print("Present:", {n: (n in globals()) for n in needed})

Present: {'top_k_semantic_neighbors_by_text': True, 'filter_by_orientation': True, 'filter_by_location': True, 'filter_by_age': True, 'filter_by_diet': True, 'filter_by_drinks': True, 'filter_by_smokes': True, 'filter_by_drugs': True, 'compute_component_scores': True, 'apply_weights': True, 'get_matches': True}


In [10]:
# Cell 2 — OkCupid-style compatibility scorer
def okcupid_score(user_id, k=20, weights=None,pool_k=5000, max_km=50,allow_missing=True,
    drinks_strict=False, smokes_strict=False, drugs_strict=True, hard_filters=None):
    """
    Returns a DataFrame of top-k matches with 0–100% 'compat_pct' plus components.

    Hard filters: orientation, location (≤ max_km), age, and optionally diet/drinks/smokes/drugs if hard_filters True.
    Soft scores:  text_sim, age_score (binary by your rule), lifestyle_score (avg), loc_score (linear decay by distance/max_km).
    """
    if weights is None:
        weights = {"text":0.70, "age":0.15, "life":0.10, "loc":0.05}
    if hard_filters is None:
        # default: similar to app OkCupid-ish — keep diet & drugs as gates, drinks/smokes as soft
        hard_filters = {"diet": True, "drinks": False, "smokes": False, "drugs": True}

    pool_k = min(int(pool_k), len(df) - 1)

    # 1) Hard-filtered candidate pool (preserves semantic order)
    cands = get_matches(user_id=user_id,k=pool_k,use_orientation=True,use_location=True, max_km=max_km,
        use_age=True, use_diet=hard_filters.get("diet", True), 
        use_drinks=hard_filters.get("drinks", False),  drinks_strict=drinks_strict,
        use_smokes=hard_filters.get("smokes", False),  smokes_strict=smokes_strict,
        use_drugs=hard_filters.get("drugs", True),     drugs_strict=drugs_strict,
        allow_missing=allow_missing
    ).copy()

    # 2) Component scores (adds text_sim/age_score/lifestyle_score/loc_score/distance_km)
    cands = compute_component_scores(user_id=user_id, cands=cands, max_km=max_km,
                                     drinks_strict=drinks_strict, smokes_strict=smokes_strict, 
                                     drugs_strict=drugs_strict,allow_missing=allow_missing
                                    )
    cands = _ensure_text_sim(cands)

    # 3) Weighted blend → 0–1, then convert to %
    ranked = apply_weights(cands, w_text=weights["text"], w_age=weights["age"], w_life=weights["life"],
                           w_loc=weights["loc"]).copy()
    ranked["compat_pct"] = (ranked["final_score"].clip(0, 1) * 100).round(1)

    # 4) Final shape
    want = ["user_id","compat_pct","text_sim","age_score","lifestyle_score","loc_score","distance_km",
            "location","age","sex","orientation"]
    for col in want:
        if col not in ranked.columns:
            ranked[col] = np.nan
    return ranked[want].head(k).reset_index(drop=True)


In [11]:
okc = okcupid_score(
    user_id=1234,
    k=10,
    weights={"text":0.70, "age":0.15, "life":0.10, "loc":0.05},
    pool_k=5000,
    max_km=50,
    allow_missing=True,
    drinks_strict=False, smokes_strict=False, drugs_strict=True,
    hard_filters={"diet": True, "drinks": False, "smokes": False, "drugs": True}
)
okc

Start pool: 5000 (max_candidates=5000)
After orientation: 2023
Unfiltered: 2023 | After location filter (≤50 km): 1717
After location (≤50 km): 1717
After age rule: 1342
Diet| before: 1342 → after: 1152
After diet: 1152
Drugs  | before: 1152 → after: 1034 (strict=True)
After drugs (strict=True): 1034


,user_id,compat_pct,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation
0,4064,64.0,0.486174,1.0,1.0,1.000000,0.000000,"palo alto, california",34,f,straight
1,57256,64.0,0.544072,1.0,1.0,0.175382,41.230915,"oakland, california",33,f,straight
2,57586,63.5,0.484042,1.0,1.0,0.916146,4.192677,"atherton, california",35,f,straight
3,39282,63.3,0.499256,1.0,1.0,0.672682,16.365892,"san mateo, california",30,f,straight
4,53843,63.2,0.533671,1.0,1.0,0.175382,41.230915,"oakland, california",47,f,straight
5,33605,63.0,0.515604,1.0,1.0,0.381272,30.936387,"castro valley, california",34,f,straight
6,4015,62.9,0.531963,1.0,1.0,0.126063,43.696864,"san francisco, california",38,f,straight
7,30039,62.9,0.531841,1.0,1.0,0.126063,43.696864,"san francisco, california",24,f,straight
8,49650,62.1,0.517111,1.0,1.0,0.175382,41.230915,"oakland, california",31,f,straight
9,20404,62.0,0.518943,1.0,1.0,0.126063,43.696864,"san francisco, california",43,f,straight


In [12]:
def add_reasons(df_in):
    df = df_in.copy()
    def _r(r):
        dist = "" if pd.isna(r.get("distance_km")) else f", ~{int(round(r.distance_km))} km"
        return f"text={r.text_sim:.2f}, age={int(r.age_score)}, life={r.lifestyle_score:.2f}, loc={r.loc_score:.2f}{dist}"
    df["reasons"] = df.apply(_r, axis=1)
    return df

add_reasons(okc).head(10)

,user_id,compat_pct,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation,reasons
0,4064,64.0,0.486174,1.0,1.0,1.000000,0.000000,"palo alto, california",34,f,straight,"text=0.49, age=1, life=1.00, loc=1.00, ~0 km"
1,57256,64.0,0.544072,1.0,1.0,0.175382,41.230915,"oakland, california",33,f,straight,"text=0.54, age=1, life=1.00, loc=0.18, ~41 km"
2,57586,63.5,0.484042,1.0,1.0,0.916146,4.192677,"atherton, california",35,f,straight,"text=0.48, age=1, life=1.00, loc=0.92, ~4 km"
3,39282,63.3,0.499256,1.0,1.0,0.672682,16.365892,"san mateo, california",30,f,straight,"text=0.50, age=1, life=1.00, loc=0.67, ~16 km"
4,53843,63.2,0.533671,1.0,1.0,0.175382,41.230915,"oakland, california",47,f,straight,"text=0.53, age=1, life=1.00, loc=0.18, ~41 km"
5,33605,63.0,0.515604,1.0,1.0,0.381272,30.936387,"castro valley, california",34,f,straight,"text=0.52, age=1, life=1.00, loc=0.38, ~31 km"
6,4015,62.9,0.531963,1.0,1.0,0.126063,43.696864,"san francisco, california",38,f,straight,"text=0.53, age=1, life=1.00, loc=0.13, ~44 km"
7,30039,62.9,0.531841,1.0,1.0,0.126063,43.696864,"san francisco, california",24,f,straight,"text=0.53, age=1, life=1.00, loc=0.13, ~44 km"
8,49650,62.1,0.517111,1.0,1.0,0.175382,41.230915,"oakland, california",31,f,straight,"text=0.52, age=1, life=1.00, loc=0.18, ~41 km"
9,20404,62.0,0.518943,1.0,1.0,0.126063,43.696864,"san francisco, california",43,f,straight,"text=0.52, age=1, life=1.00, loc=0.13, ~44 km"
